# PAGES_C-PEAT Data Retrieval and Event Mapping

In [ ]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
from collections import Counter

In [ ]:
### PANGAEApy
## if you need to install PANGAEApy use pip
#!pip install pangaeapy # Uncomment to install pangaeapy

## if you need to upgrade PANGAEApy use 
#!pip install pangaeapy --upgrade # Uncomment to upgrade pangaeapy

## check version of PANGAEApy
!pip show pangaeapy

## for details on PANGAEApy see https://pypi.org/project/pangaeapy/ 

import pangaeapy as pan
from pangaeapy.pandataset import PanDataSet

In [ ]:
### ignore warnings in this script
import warnings
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=(SettingWithCopyWarning))
warnings.simplefilter(action='ignore', category=FutureWarning)

## Search PAGES_C-PEAT records

In [ ]:
### Get all results and combine them in data frame.

### define search pattern
search_pattern = 'project:label:PAGES_C-PEAT'

### basic query to get number of search results
query = pan.PanQuery(search_pattern, limit = 500)

### create empty data frame
df_PAGES = pd.DataFrame()

### loop over all results in steps of 500
for i in np.arange(0,query.totalcount,500):
    
    ### store result of individual step in qs
    qs = pan.PanQuery(search_pattern, limit = 500, offset=i)
    
    ### convert qs result with 500 entries to data frame df_qs
    df_qs = pd.DataFrame(qs.result)
    
    ### concatenate all individual df_qs into one data frame named query_results_all
    df_PAGES = pd.concat([df_PAGES,df_qs],ignore_index=True)


## Get Metadata

In [ ]:
### Loop over all entries in df and get metadata for each entry

# NOTE: As a safety precaution, the number of metadata requests is limited for a specific time period. 
# _Received too many (metadata) requests error (429)...waiting 30s -_

# If you have larger requests, prepare to wait or use a different tool e.g. OAI-PMH (https://wiki.pangaea.de/wiki/OAI-PMH).

for ind,value in df_PAGES['URI'].items():    
    ## use PanDataSet to get metadata 
    ds = PanDataSet(value, include_data=False)
    # print(ind, ds.doi)
    ## put metadata into df in new columns
    df_PAGES.loc[ind,'Citation'] = ds.citation
    df_PAGES.loc[ind,'DOI'] = ds.uri
    df_PAGES.loc[ind,'PANGAEA ID'] = ds.id
    df_PAGES.loc[ind,'Title'] = ds.title
     
    if ds.events:
        df_PAGES.loc[ind,'event label'] = "; ".join([x.label for x in ds.events])

df_PAGES_citation = df_PAGES[['Citation','DOI']]

### write citations as txt file
date_today = datetime.today().strftime('%Y-%m-%d')
df_PAGES_citation.to_csv('citations_PAGES_'+date_today+'.txt',index=False,sep='\t')

## Split metadata by type
get actual data later

In [ ]:
df_PAGES_age  = df_PAGES[df_PAGES['Title'].str.startswith('Age determination')]
df_PAGES_geochem  = df_PAGES[df_PAGES['Title'].str.startswith('Geochemistry')]
df_PAGES_cal  = df_PAGES[df_PAGES['Title'].str.startswith('Calibrated ages')]

In [ ]:
### sort PANGAEA id numbers by event
df_events_list = df_PAGES['event label'].unique()
df_events = pd.DataFrame(list(zip(df_events_list)), columns = ['event'])
# df_events_list
df_events.insert(loc=1,column='age',value=np.nan)
df_events.insert(loc=2,column='cal_age',value=np.nan)
df_events.insert(loc=3,column='geochem',value=np.nan)

In [ ]:
for ind,value in df_PAGES['event label'].items():
    for ind2,value2 in df_events['event'].items():
        if value==value2:
            if df_PAGES.loc[ind,'Title'].startswith('Calibrated ages'):
                df_events.loc[ind2,'cal_age'] = df_PAGES.loc[ind,'PANGAEA ID']
            elif  df_PAGES.loc[ind,'Title'].startswith('Geochemistry'):
                df_events.loc[ind2,'geochem'] = df_PAGES.loc[ind,'PANGAEA ID']
            elif  df_PAGES.loc[ind,'Title'].startswith('Age determination'):
                df_events.loc[ind2,'age'] = df_PAGES.loc[ind,'PANGAEA ID']
               
df_events.to_csv('dataset_IDs_by_events_'+date_today+'.txt',index=False,sep='\t')

In [ ]:
df_events.head()

## Download data of individual types

### Helping functions

In [ ]:
### Translate short parameters names to long names including unit
def get_long_parameters(ds):
    """Translate short parameters names to long names including unit

    Args:
        ds (PANGAEA dataset): PANGAEA dataset
    """
    ds.data.columns =  [f'{param.name} [{param.unit}]' if param.unit else param.name for param in ds.params.values()]

**Practical functions for complicated datasets**  
* double parameter
* method as comment  

Example dataset: https://doi.pangaea.de/10.1594/PANGAEA.890478  

In [ ]:
### function to find duplicate column names
def find_duplicates(col_names):
    name_counts = Counter(col_names)
    duplicates = [name for name, count in name_counts.items() if count > 1]
    return duplicates, bool(duplicates)

In [ ]:
### function to create general parameter long names with unit and method
def generate_general_column_name(param):
    base_name = f'{param.name} [{param.unit}]' if param.unit else param.name 

    if param.method:
        base_name += f', method:{param.method.name}'

    return base_name

In [ ]:
### functions to rename duplicate column names so they are all individual within the dataset
def generate_unique_column_name(param):
    base_name = f'{param.name} [{param.unit}]' if param.unit else param.name
    
    if param.method:
        base_name += f', method:{param.method.name}'
    
    if param.comment:
        return f'{base_name}, comment:{param.comment}'
    else:
        return f'{base_name}, col nr:{param.colno}'

def make_unique_column_names(ds, same_param_name):
    col_names = []
    for param in ds.params.values():
        name = generate_general_column_name(param)
        if name in same_param_name:
            name = generate_unique_column_name(param)
        col_names.append(name)
    return col_names

### Geochemistry datasets

In [ ]:
df_PAGES_geochem.head(3)

In [ ]:
### Create one data frame for all datasets
df_PAGES_geochem_data = pd.DataFrame()

### loop over all datasets in df_PAGES_geochem
for ind,value in df_PAGES_geochem['URI'].items():
    
    ## use PanDataSet to get metadata and data and put them into 2 diferent dataframes
    ds = PanDataSet(value)

    print(ind, ds.doi)
    
    ### Translate default short parameter names to long parameter names, add unit and method if available, check if all column names are individuals
    col_names = []
    for param in ds.params.values():
        col_name = generate_general_column_name(param)
        col_names.append(col_name)

    ### find duplicate column names make them individual column names
    same_param_name, double_name = find_duplicates(col_names)

    if double_name:
        col_names = make_unique_column_names(ds, set(same_param_name))
    
    ### rename columns because python cannot handle duplicate column names within dataframe
    ds.data.columns =  col_names
    
    ### create new data dataframe for each query result 
    df_data = pd.DataFrame()
    df_data = ds.data
    df_data['DOI'] = ds.doi
    if ds.events:
        df_data['Event label'] = "; ".join([x.label for x in ds.events])
        
    ### combine all datasats into one dataframe
    df_PAGES_geochem_data = pd.concat([df_PAGES_geochem_data,df_data], ignore_index=True)

### save data
# df_PAGES_geochem_data.to_csv('../Data/geochem_data_all_'+date_today+'.txt', sep='\t', encoding='utf-8', index=False)
df_PAGES_geochem_data.to_csv('geochem_data_all_'+date_today+'.txt', sep='\t', encoding='utf-8', index=False)

### Calibrated Ages datasets

In [ ]:
df_PAGES_cal.head(3)

In [ ]:
### Create one data frame for all datasets
df_PAGES_cal_data = pd.DataFrame()

### loop over all datasets in df_PAGES_geochem
for ind,value in df_PAGES_cal['URI'].items():
    
    ## use PanDataSet to get metadata and data and put them into 2 diferent dataframes
    ds = PanDataSet(value)

    print(ind, ds.doi)
    
    ### Translate default short parameter names to long parameter names, add unit and method if available, check if all column names are individuals
    col_names = []
    for param in ds.params.values():
        col_name = generate_general_column_name(param)
        col_names.append(col_name)

    ### find duplicate column names make them individual column names
    same_param_name, double_name = find_duplicates(col_names)

    if double_name:
        col_names = make_unique_column_names(ds, set(same_param_name))
    
    ### rename columns because python cannot handle duplicate column names within dataframe
    ds.data.columns =  col_names
    
    ### create new data dataframe for each query result 
    df_data = pd.DataFrame()
    df_data = ds.data
    df_data['DOI'] = ds.doi
    if ds.events:
        df_data['Event label'] = "; ".join([x.label for x in ds.events])
        
    ### combine all datasats into one dataframe
    df_PAGES_cal_data = pd.concat([df_PAGES_cal_data,df_data], ignore_index=True)

### save data
df_PAGES_cal_data.to_csv('cal_age_data_all_'+date_today+'.txt', sep='\t', encoding='utf-8', index=False)